In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from salishsea_tools import evaltools as et, viz_tools
import os
import scipy.interpolate as sinterp
import cmocean
import netCDF4 as nc
import xarray as xr

In [2]:
path_run = '/results2/SalishSea/nowcast-green.202111' 
data = pd.read_csv('/home/jvalenti/MOAD/analysis-jose/notebooks/SHEM/eval/SalishSea_MesozoopModelvsObsMatches_2015to2019_Eval202111.csv', index_col=0,header=0)

In [3]:
start_date = datetime(2018,1,1)
end_date = datetime(2018,12,31)
flen=1
namfmt='nowcast'
filemap={'nitrate':'ptrc_T','silicon':'ptrc_T','ammonium':'ptrc_T','diatoms':'ptrc_T',
         'ciliates':'ptrc_T','flagellates':'ptrc_T','vosaline':'grid_T','votemper':'grid_T'}
fdict={'ptrc_T':1,'grid_T':1}

df1=et.loadDFO(datelims=(start_date,end_date))
print(len(df1),'data points')
df1[['Year','Month','Day','Lat','Lon','Pressure','Depth','N','Si','Chlorophyll_Extracted',
     'ConsT','AbsSal']].head()

775 data points


,Year,Month,Day,Lat,Lon,Pressure,Depth,N,Si,Chlorophyll_Extracted,ConsT,AbsSal
0,2018.0,3.0,6.0,48.299167,-124.003667,2.3,2.3,25.04,43.58,NaN,7.772878,31.863860
1,2018.0,3.0,6.0,48.299167,-124.003667,2.3,2.3,NaN,NaN,1.59,7.773274,31.864262
2,2018.0,3.0,6.0,48.299167,-124.003667,2.3,2.3,NaN,NaN,NaN,7.776051,31.866473
3,2018.0,3.0,6.0,48.299167,-124.003667,5.1,5.1,25.57,43.75,NaN,7.769256,31.873879
4,2018.0,3.0,6.0,48.299167,-124.003667,10.0,9.9,NaN,NaN,NaN,7.775262,31.884705


In [4]:
df1.to_excel('nutrients_2018dfo.xlsx')

In [ ]:
def make_filename(path_run,start,day=0, var='biol_T', res='d'):
    """Construct path prefix for local SHEM results given date object and paths dict
    """
    date  =  (start + timedelta(days=day))
    folder = date.strftime("%d%b%y").lower()
    prefix = os.path.join(path_run, f'{folder}/')
    fname = []
    for file in os.listdir(prefix):
        if (var in file) and ('_1'+res) in file:
            fname.append(file)
    if len(fname)>1:
        print('more than one file found') 
    
    return os.path.join(prefix, fname[0])

In [ ]:
M_path = '/home/jvalenti/MOAD/'
mask = xr.open_dataset(M_path+'grid2/mesh_mask202108_TDV.nc')

In [ ]:
data['mod_mesozooplankton']

0       0.342959
1       0.247124
2       0.321169
3       0.218180
4       0.233768
          ...   
2059    0.405539
2060    0.404237
2061    0.408658
2062    0.362514
2063    0.514349
Name: mod_mesozooplankton, Length: 2064, dtype: float64

In [ ]:
example = data.iloc[4]
start = datetime.strptime(example.dtUTC, '%Y-%m-%d %H:%M:%S')
sum(np.array(xr.open_dataset(make_filename(path_run,start)).mesozooplankton[0,example.k_upper:example.k_lower+1,example.j,example.i])*np.array(mask.e3t_0[0,example.k_upper:example.k_lower+1,example.j,example.i])*np.array(mask.tmask[0,example.k_upper:example.k_lower+1,example.j,example.i]))/(mask.e3t_0[0,example.k_upper:example.k_lower+1,example.j,example.i]*np.array(mask.tmask[0,example.k_upper:example.k_lower+1,example.j,example.i])).sum()
#*12*5.7/1000

<xarray.DataArray 'e3t_0' ()> Size: 8B
array(0.21926726)

In [ ]:
total_mine = np.zeros(len(data))
for n in range(len(data)):
    example = data.iloc[n]
    start = datetime.strptime(example.dtUTC, '%Y-%m-%d %H:%M:%S')
    jj = example.j
    ii = example.i
    k_u = example.k_upper
    k_l = example.k_lower
    e3tt = np.array(mask.e3t_0[0,k_u:k_l+1,jj,ii])
    masktt = np.array(mask.tmask[0,k_u:k_l+1,jj,ii])
    total_mine[n] = sum(np.array(xr.open_dataset(make_filename(path_run,start)).mesozooplankton[0,k_u:k_l+1,jj,ii])*e3tt*masktt)/sum(e3tt*masktt)

KeyboardInterrupt: 